In [1]:
# imports an excel sheet of ConceptualEntity <> ConceptualEntity
# expects the Conceptual Entities to be fully qualified.
import sqlite3
import pandas as pd
import sys
sys.path.append('../')
from user_packages import hashing

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

TargetExcelFilename = 'ConceptualEntityConceptualEntity_Related.xlsx'
TargetExcelFilepath = 'imports/concept/'

In [2]:
df = pd.read_excel(TargetExcelFilepath + TargetExcelFilename)
df.fillna('', inplace=True)

df['ConceptualEntityKeyPhrase'] = df.apply(lambda row: row['type'].strip() + '.' + row['name'].strip(), axis=1)
#df

node_dict = df.set_index('id').to_dict('index')
#print(node_dict)

In [7]:
df = pd.read_excel(TargetExcelFilepath + TargetExcelFilename, 'edges')
df.fillna('', inplace=True)

df['RecordSource'] = df.apply(lambda row: 'Py.XL_CE_to_CERelated', axis=1)

df['ConceptualEntityKeyPhrase'] = df.apply(lambda row: node_dict[row['ConceptualEntity']]['ConceptualEntityKeyPhrase'], axis=1)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntityHashKey'
  , columns = ['ConceptualEntityKeyPhrase']
)

df['ConceptualEntity_RelatedKeyPhrase'] = df.apply(lambda row: node_dict[row['ConceptualEntity_Related']]['ConceptualEntityKeyPhrase'], axis=1)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntity_RelatedHashKey'
  , columns = ['ConceptualEntity_RelatedKeyPhrase']
)

df['ConceptualEntityConceptualEntity_RelatedKeyPhrase'] = df.apply(lambda row: row['ConceptualEntityKeyPhrase'] + ':' + row['ConceptualEntity_RelatedKeyPhrase'], axis=1)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ConceptualEntityConceptualEntity_RelatedHashKey'
  , columns = ['ConceptualEntityConceptualEntity_RelatedKeyPhrase']
)

df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['label_forward', 'isDeleted']
)

df = df.drop(columns=['ConceptualEntity', 'ConceptualEntity_Related'])

df

,label_forward,isDeleted,RecordSource,ConceptualEntityKeyPhrase,ConceptualEntityHashKey,ConceptualEntity_RelatedKeyPhrase,ConceptualEntity_RelatedHashKey,ConceptualEntityConceptualEntity_RelatedKeyPhrase,ConceptualEntityConceptualEntity_RelatedHashKey,HashDiff
0,part of,0,Py.XL_CE_to_CERelated,wheel.count.wash.Case,743e68daa8771750b0a2cd47754aac4a,wheel.count.wash.Sale,d7169efc6ef5e7def3ba661d43fd5e14,wheel.count.wash.Case:wheel.count.wash.Sale,f452ed2d9d22f3119fdf50925611ebe8,30acf775b0768eb03f436337ed61e517
1,makes a,0,Py.XL_CE_to_CERelated,wheel.count.wash.Seller (Person),4e2fa42a64a12e1f0767a05d8811a72f,wheel.count.wash.Sale,d7169efc6ef5e7def3ba661d43fd5e14,wheel.count.wash.Seller (Person):wheel.count.w...,ddca807ce1a5f6f324f5dff009925035,14a19e7ddeb4cf3caf634866c6e3e2d2
2,buys a,0,Py.XL_CE_to_CERelated,wheel.count.wash.Client (Person),5531ce268d2c0155098e29e827b76fa2,wheel.count.wash.Sale,d7169efc6ef5e7def3ba661d43fd5e14,wheel.count.wash.Client (Person):wheel.count.w...,d1fc9341f9475097d162ed78163d01b4,f3346eb7f3455944712f34097be0f45f
3,contains a,0,Py.XL_CE_to_CERelated,wheel.count.wash.Case,743e68daa8771750b0a2cd47754aac4a,wheel.count.wash.Product,fb025572ad6bfea5bd6d3a6ce0ab9798,wheel.count.wash.Case:wheel.count.wash.Product,735189354ba814d302a33946481f4d9f,d223fd0ccbccec7ff37b6719a29e89d2
4,completes a,0,Py.XL_CE_to_CERelated,wheel.count.wash.Client (Person),5531ce268d2c0155098e29e827b76fa2,wheel.count.wash.KYC,8e04423620da3a67f1fa4a8929f621e2,wheel.count.wash.Client (Person):wheel.count.w...,790b01be241c011a7c0a046e75e6f901,fb6a8f8504f9902f0c795338ab8b6598
...,...,...,...,...,...,...,...,...,...,...
90,levied against,0,Py.XL_CE_to_CERelated,wheel.count.wash.Remedial Action,a7b1cbf1a1654a133a1fcfc0e12de3ad,wheel.count.wash.Adviser (Person),a90b8acc88ea4fd3ee8bf96e763b5f77,wheel.count.wash.Remedial Action:wheel.count.w...,4a4da2ea6a2d8c97f03ac715f192ceee,aa7f0c24f4ea4e5dc366a0fcd6bd5075
91,reported to,0,Py.XL_CE_to_CERelated,wheel.count.wash.Adviser Quality,2409452767e8d341045ab867d4856dd0,wheel.count.wash.Adviser (Person),a90b8acc88ea4fd3ee8bf96e763b5f77,wheel.count.wash.Adviser Quality:wheel.count.w...,444c07bc9f66b2bac41dad45de2c411b,22102282447872d581c695f1fb831bb4
92,reported to,0,Py.XL_CE_to_CERelated,wheel.count.wash.Breach,ffbfa9193a13fb7af850072710460eef,wheel.count.wash.Regulatory Body,1e524bff40140d097bedf4a6a0807236,wheel.count.wash.Breach:wheel.count.wash.Regul...,f1b1ed41ab3c9ec63f05e3943881329c,22102282447872d581c695f1fb831bb4
93,provides framework for,0,Py.XL_CE_to_CERelated,wheel.count.wash.Adviser Quality,2409452767e8d341045ab867d4856dd0,wheel.count.wash.Supervision,21442e20a5e760eb398c01a54474f7c9,wheel.count.wash.Adviser Quality:wheel.count.w...,877ae92c167cdd3e1b1f381f512fb66f,cb4778e2155bdf9dfdaa608c73b92d0b


In [8]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_XL_CE_to_CERelated")
conn.commit()
df.to_sql('stg_Py_XL_CE_to_CERelated', conn, if_exists='append', index=False)
conn.commit()

95

In [10]:
# update DeletedDate in staging table if anything is deleted
conn.execute("UPDATE stg_Py_XL_CE_to_CERelated SET DeletedDate = datetime() WHERE isDeleted = 1")
conn.commit()

In [11]:
# Write to the Link (ConceptualEntity - ConceptualEntity_Related)
sql_query = """
INSERT INTO rv_l_ConceptualEntityConceptualEntity_Related
(
  ConceptualEntityConceptualEntity_RelatedHashKey
  , LoadDate
  , RecordSource
  , ConceptualEntityHashKey
  , ConceptualEntity_RelatedHashKey
)
SELECT
  stg.ConceptualEntityConceptualEntity_RelatedHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.ConceptualEntityHashKey
  , stg.ConceptualEntity_RelatedHashKey
FROM
  stg_Py_XL_CE_to_CERelated stg
WHERE
  stg.ConceptualEntityConceptualEntity_RelatedHashKey IS NOT NULL
  AND ConceptualEntityConceptualEntity_RelatedHashKey NOT IN (
    SELECT ConceptualEntityConceptualEntity_RelatedHashKey
    FROM rv_l_ConceptualEntityConceptualEntity_Related
  )
""";
conn.execute(sql_query)
conn.commit()

In [13]:
# Write to the Satellite (rv_s_ConceptualEntityConceptualEntity_Related)
sql_query = """
INSERT INTO rv_s_ConceptualEntityConceptualEntity_Related
(
    ConceptualEntityConceptualEntity_RelatedHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , label_forward
  , isDeleted
  , DeletedDate
)
SELECT DISTINCT
    stg.ConceptualEntityConceptualEntity_RelatedHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.label_forward
  , stg.isDeleted
  , stg.DeletedDate
FROM
  stg_Py_XL_CE_to_CERelated AS stg
  LEFT OUTER JOIN rv_s_ConceptualEntityConceptualEntity_Related AS sat ON (
    stg.ConceptualEntityConceptualEntity_RelatedHashKey = sat.ConceptualEntityConceptualEntity_RelatedHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_ConceptualEntityConceptualEntity_Related AS z
      WHERE z.ConceptualEntityConceptualEntity_RelatedHashKey = sat.ConceptualEntityConceptualEntity_RelatedHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()